In [1]:
import torch.nn as nn
import torch
from transformers import BertModel, BertTokenizer, BertConfig
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [2]:
USE_CUDA=torch.cuda.is_available()
if(USE_CUDA):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [3]:
import torch.nn as nn
import torch
from transformers import BertModel, BertTokenizer, BertConfig

class bert_lstm(nn.Module):
    def __init__(self, hidden_dim,output_size,n_layers,bidirectional=True, drop_prob=0.5):
        super(bert_lstm, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.bidirectional = bidirectional

        #Bert ----------------重点，bert模型需要嵌入到自定义模型里面
        self.bert=BertModel.from_pretrained('/Users/jasondennis/Desktop/chinese_L-12_H-768_A-12')
        for param in self.bert.parameters():
            param.requires_grad = True

        # LSTM layers
        self.lstm = nn.LSTM(768, hidden_dim, n_layers, batch_first=True,bidirectional=bidirectional)

        # dropout layer
        self.dropout = nn.Dropout(drop_prob)

        # linear and sigmoid layers
        if bidirectional:
            self.fc = nn.Linear(hidden_dim*2, output_size)
        else:
            self.fc = nn.Linear(hidden_dim, output_size)

        #self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        batch_size = x.size(0)
        #生成bert字向量
        x=self.bert(x)[0]     #bert 字向量

        # lstm_out
        #x = x.float()
        lstm_out, (hidden_last,cn_last) = self.lstm(x, hidden)
        #print(lstm_out.shape)   #[32,100,768]
        #print(hidden_last.shape)   #[4, 32, 384]
        #print(cn_last.shape)    #[4, 32, 384]

        #修改 双向的需要单独处理
        if self.bidirectional:
            #正向最后一层，最后一个时刻
            hidden_last_L=hidden_last[-2]
            #print(hidden_last_L.shape)  #[32, 384]
            #反向最后一层，最后一个时刻
            hidden_last_R=hidden_last[-1]
            #print(hidden_last_R.shape)   #[32, 384]
            #进行拼接
            hidden_last_out=torch.cat([hidden_last_L,hidden_last_R],dim=-1)
            #print(hidden_last_out.shape,'hidden_last_out')   #[32, 768]
        else:
            hidden_last_out=hidden_last[-1]   #[32, 384]


        # dropout and fully-connected layer
        out = self.dropout(hidden_last_out)
        #print(out.shape)    #[32,768]
        out = self.fc(out)

        return out

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data

        number = 1
        if self.bidirectional:
            number = 2

        if (USE_CUDA):
            hidden = (weight.new(self.n_layers*number, batch_size, self.hidden_dim).zero_().float().cuda(),
                      weight.new(self.n_layers*number, batch_size, self.hidden_dim).zero_().float().cuda()
                      )
        else:
            hidden = (weight.new(self.n_layers*number, batch_size, self.hidden_dim).zero_().float(),
                      weight.new(self.n_layers*number, batch_size, self.hidden_dim).zero_().float()
                      )

        return hidden

In [4]:
output_size = 1
hidden_dim = 384   #768/2
n_layers = 2
bidirectional = True  #这里为True，为双向LSTM

net = bert_lstm(hidden_dim, output_size,n_layers, bidirectional)

print(net)
    

Some weights of the model checkpoint at /Users/jasondennis/Desktop/chinese_L-12_H-768_A-12 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


bert_lstm(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

In [5]:
np.random.seed(2020)
torch.manual_seed(2020)

In [6]:
data=pd.read_csv('/Users/jasondennis/Desktop/train_data.csv',encoding='utf-8')


In [7]:
def pretreatment(comments):
    result_comments=[]
    punctuation='。，？！：%&~（）、；“”&|,.?!:%&~();""'
    for comment in comments:
        comment= ''.join([c for c in comment if c not in punctuation])
        comment= ''.join(comment.split())   #\xa0
        result_comments.append(comment)
    
    return result_comments

In [8]:
result_comments=pretreatment(list(data['text_a'].values))

In [9]:
len(result_comments)

73928

In [10]:
result_comments[:1]

['钢铁侠那神一样的吐槽']

In [11]:
from transformers import BertTokenizer,BertModel
tokenizer = BertTokenizer.from_pretrained('/Users/jasondennis/Desktop/chinese_L-12_H-768_A-12')

In [12]:
result_comments_id=tokenizer(result_comments,padding=True,truncation=True,max_length=200,return_tensors='pt')



In [13]:
result_comments_id


{'input_ids': tensor([[ 101, 7167, 7188,  ...,    0,    0,    0],
        [ 101, 2600, 1377,  ...,    0,    0,    0],
        [ 101, 4035, 2014,  ...,    0,    0,    0],
        ...,
        [ 101, 3241, 5688,  ...,    0,    0,    0],
        [ 101, 2523, 3300,  ...,    0,    0,    0],
        [ 101, 5629, 3553,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X=result_comments_id['input_ids']
y=torch.from_numpy(data['label'].values).float()

X_train,X_test, y_train, y_test =train_test_split(X,y,test_size=0.3,shuffle=True,stratify=y,random_state=2020)

In [16]:
len(X_train),len(X_test)

(51749, 22179)

In [17]:
X_train.shape

torch.Size([51749, 200])

In [18]:
X_valid,X_test,y_valid,y_test=train_test_split(X_test,y_test,test_size=0.5,shuffle=True,stratify=y_test,random_state=2020)

In [19]:
train_data = TensorDataset(X_train, y_train)
valid_data = TensorDataset(X_valid, y_valid)
test_data = TensorDataset(X_test,y_test)

# dataloaders
batch_size = 64

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size,drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,drop_last=True)

In [20]:
# loss and optimization functions
lr=2e-5
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

# training params
epochs = 10
# batch_size=50
print_every = 7
clip=5 # gradient clipping
 
# move model to GPU, if available
if(USE_CUDA):
    net.cuda()

In [ ]:
net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)
    counter = 0
 
    # batch loop
    for inputs, labels in train_loader:
        counter += 1
        
        if(USE_CUDA):
            inputs, labels = inputs.cuda(), labels.cuda()
        h = tuple([each.data for each in h])
        net.zero_grad()
        output= net(inputs, h)
        print(output.squeeze())
        print(labels.float())
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
 
        # loss stats
        if counter % print_every == 0:
            net.eval()
            with torch.no_grad():
                val_h = net.init_hidden(batch_size)
                val_losses = []
                for inputs, labels in valid_loader:
                    val_h = tuple([each.data for each in val_h])

                    if(USE_CUDA):
                        inputs, labels = inputs.cuda(), labels.cuda()

                    output = net(inputs, val_h)
                    val_loss = criterion(output.squeeze(), labels.float())

                    val_losses.append(val_loss.item())
 
            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

tensor([-0.0345, -0.0819, -0.0456, -0.1099, -0.0755, -0.0330, -0.1172,  0.0218,
        -0.0748, -0.0657, -0.0667, -0.0286, -0.0150, -0.0701,  0.0696, -0.0410,
         0.0654,  0.0344,  0.0036, -0.0614, -0.0180, -0.0444, -0.0135, -0.0221,
         0.0130, -0.0599,  0.0061, -0.1090,  0.0237, -0.0041, -0.0857, -0.0533,
        -0.0357, -0.0462, -0.0220,  0.0007,  0.0162,  0.0481, -0.0484,  0.0262,
        -0.0499, -0.0486,  0.0088,  0.0595, -0.0791, -0.1440,  0.0057, -0.0915,
        -0.0183, -0.0221, -0.0354, -0.1021, -0.0359, -0.1043,  0.0454, -0.0266,
        -0.0529,  0.0826, -0.0058, -0.0509, -0.0633, -0.0694,  0.0440,  0.0089],
       grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
        1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0.,
        1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 1., 1., 1.])
tensor([0.1982, 0.3363, 0.2334, 0.1

tensor([1.0452, 1.1375, 1.1311, 0.8804, 1.0785, 1.0545, 0.8999, 1.1142, 1.1707,
        1.0791, 0.9347, 1.0955, 1.0688, 1.0898, 1.0520, 1.0127, 1.0768, 1.0543,
        1.0596, 1.0805, 0.9896, 0.9946, 1.0537, 1.2381, 1.0162, 1.0402, 1.1667,
        1.0407, 0.9767, 0.9944, 1.0406, 1.1545, 1.1426, 1.0699, 1.2172, 1.0168,
        0.8857, 1.0699, 1.0239, 1.0623, 1.0267, 0.9317, 0.9354, 1.1519, 0.9743,
        1.3682, 0.9983, 1.1219, 1.0292, 1.1475, 0.9730, 1.1027, 1.0642, 0.9811,
        0.9491, 1.1287, 1.0356, 0.8321, 1.0759, 0.8515, 1.1650, 0.8952, 1.1733,
        1.1265], grad_fn=<SqueezeBackward0>)
tensor([1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1.,
        1., 1., 0., 1., 1., 0., 1., 1., 1., 1.])
tensor([1.1602, 1.0659, 1.2593, 1.1328, 1.0175, 1.1664, 1.0378, 1.1780, 1.0241,
        1.0802, 1.1631, 1.

tensor([1.5183, 1.3841, 1.4094, 1.4684, 1.5277, 1.4612, 1.3629, 1.4470, 1.4718,
        1.4740, 1.2147, 1.4543, 1.4462, 1.2246, 1.2904, 1.2992, 1.4925, 1.4473,
        1.3526, 1.1663, 1.3230, 1.2694, 1.3982, 1.5523, 1.2860, 1.5175, 1.2644,
        1.6405, 1.3094, 1.5744, 1.4219, 1.5647, 1.1986, 1.4100, 1.3897, 1.4038,
        1.3682, 0.9908, 1.2542, 1.3081, 1.5699, 1.2897, 1.3181, 1.3229, 1.1383,
        1.4286, 1.2498, 1.5960, 1.5760, 1.2128, 1.5046, 1.4330, 1.4693, 1.5161,
        1.5626, 1.4474, 1.4806, 1.4613, 1.2509, 1.4234, 1.4185, 1.1339, 1.3536,
        1.4371], grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1.,
        1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
        0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 0., 1., 1., 1., 1.])
Epoch: 1/10... Step: 21... Loss: 0.552396... Val Loss: 0.507881
tensor([1.5244, 1.3886, 1.6541, 1.4461, 1.

tensor([1.1878, 1.2581, 1.1607, 0.9755, 1.0833, 1.1806, 1.2574, 1.5484, 1.1214,
        1.3183, 1.4539, 1.0134, 0.9947, 1.4457, 1.3888, 1.0125, 1.3171, 0.9699,
        1.3344, 1.4472, 1.6114, 1.2190, 1.4180, 1.5366, 1.0673, 1.1360, 1.1980,
        1.0791, 1.4580, 1.3241, 1.1758, 1.2173, 1.3202, 1.2683, 1.3219, 0.9276,
        1.1395, 1.2566, 1.4079, 1.2615, 1.4459, 1.3341, 1.4619, 1.4727, 1.1102,
        1.0449, 1.3631, 1.3594, 1.1171, 1.2808, 1.2881, 0.9477, 1.2832, 1.5679,
        1.1896, 1.2451, 1.2593, 1.1566, 1.1987, 1.2608, 0.9039, 1.2653, 1.4107,
        1.2083], grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
        1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([1.1204, 1.1603, 1.3951, 0.9540, 1.5952, 1.2611, 1.5024, 1.5232, 1.3097,
        1.5820, 1.2949, 1.

tensor([1.8396, 0.5119, 1.6891, 0.6553, 1.7857, 1.8634, 0.7188, 1.8657, 0.8313,
        1.5650, 0.7403, 0.7296, 0.7428, 0.5472, 1.8530, 0.7411, 2.0855, 0.7845,
        1.5713, 1.8079, 1.8256, 1.9139, 0.5136, 1.7959, 0.5005, 0.5166, 0.7300,
        0.6376, 0.5988, 1.6945, 1.8353, 0.5550, 0.7206, 1.6980, 0.7383, 1.6966,
        0.3887, 0.5497, 0.5004, 1.7895, 1.4752, 0.6957, 1.8270, 0.5732, 1.9852,
        1.9285, 0.6797, 0.8403, 0.5275, 0.5623, 0.6769, 1.4654, 0.4512, 1.3012,
        1.5644, 0.6691, 0.5761, 0.6031, 1.6200, 1.8660, 0.5265, 1.5308, 1.8714,
        1.3679], grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0.,
        1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
        0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0.,
        1., 0., 1., 1., 1., 1., 1., 1., 1., 0.])
tensor([1.8835, 2.0618, 1.9147, 1.8988, 0.6464, 1.6866, 1.8444, 1.6408, 1.7407,
        1.8612, 0.4234, 1.

tensor([ 2.1561,  2.1300,  1.1869,  1.7694,  2.1019,  1.0652,  1.7080,  2.4524,
         1.9251,  2.0493,  2.0683,  2.0363,  1.9046,  2.0281,  1.9009,  2.2375,
         0.2507,  0.7714,  2.1985,  2.1329, -0.1025,  1.8159,  1.7851,  2.3215,
         0.2159, -0.1210,  2.1150,  0.0083, -0.0056,  1.9653,  1.8656,  2.3862,
         0.0427,  2.1083,  2.1249,  0.0868,  0.7776,  1.8867,  1.9067,  1.7859,
         2.1615,  2.2031,  0.1209,  0.9675,  1.8982,  0.1694,  1.9698,  0.1856,
         1.7603,  0.0229,  0.5200,  0.0656,  0.2545,  2.0032,  0.0874,  0.0942,
         0.2553,  1.9845,  0.2409,  1.9154,  2.0462,  1.2178,  1.7857,  1.6946],
       grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0.,
        1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
        0., 0., 0., 1., 1., 1., 0., 0., 1., 1.])
tensor([ 1.6202e+00,  5.4342e-01,  

tensor([ 2.0434,  2.0405,  0.7147,  2.2792,  2.2788,  2.1573,  0.3289,  1.9625,
         2.0689,  2.0444,  1.9662, -0.2225,  2.1527,  1.9514,  1.9117, -0.0170,
         2.3208,  2.0960,  2.0479,  2.2640,  1.0552,  2.0116,  2.0646, -0.2180,
        -0.4563,  2.1997,  2.1444,  2.0696,  2.2808,  2.2585,  1.4745,  2.1334,
         2.2308,  1.9415,  1.3723,  1.6975,  2.0362,  1.9333,  2.0724,  1.3240,
        -0.4576,  2.2149,  2.0255,  1.6636,  1.7289, -0.3593,  2.0998,  1.8836,
         1.9378,  2.2868,  0.3530,  2.0756, -0.5005, -0.0210,  2.1048,  2.0591,
         2.0020, -0.3325,  2.0113,  2.3459,  2.1605,  2.0290,  2.0102,  1.8654],
       grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0.,
        1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0.,
        1., 1., 1., 0., 1., 1., 1., 1., 1., 1.])
tensor([-0.6718,  1.9748,  0.9272, 

tensor([ 8.7038e-01,  2.7784e+00,  2.4821e+00, -4.8174e-01, -5.6355e-01,
         2.6089e+00,  2.0916e+00,  2.3716e+00,  2.4316e+00,  2.0231e+00,
         2.1894e+00,  2.5585e+00,  2.6308e+00,  2.3610e+00,  2.4241e+00,
         2.5377e+00,  9.8402e-01,  2.7593e+00,  1.3573e+00,  2.4855e+00,
         2.4996e+00,  4.8642e-01, -7.9908e-01,  2.7476e+00, -9.1234e-01,
         2.3440e+00,  2.3851e+00,  2.3770e+00,  2.2897e+00,  2.4740e+00,
        -2.7088e-02,  2.4408e+00,  2.6068e+00,  2.4701e+00,  2.8261e-01,
         6.2558e-01, -7.2014e-01,  2.4024e+00,  2.4821e+00, -2.3380e-01,
         2.4694e+00,  2.3344e+00,  2.4874e+00,  1.4114e+00,  2.3853e+00,
         2.4961e+00,  2.3138e+00,  2.2474e+00,  2.3892e+00,  2.4512e+00,
         2.5254e+00,  2.4369e+00,  2.5391e+00, -4.5851e-04,  2.8092e+00,
         2.2838e+00, -7.5108e-01,  2.5988e+00,  2.3439e+00,  2.3698e+00,
        -8.7358e-01,  1.2724e+00,  2.4258e+00, -6.6628e-01],
       grad_fn=<SqueezeBackward0>)
tensor([1., 1., 0., 0., 1., 

tensor([ 2.7433,  1.3484,  2.0775,  2.4664,  2.6848,  1.8957, -1.3357, -1.0546,
         2.7189,  1.7433,  2.3353,  2.9370,  2.8686, -0.8018, -0.7741,  2.4912,
         2.7605,  2.6625, -0.2679,  2.9080,  2.0813,  0.5851,  2.2065,  2.5942,
         2.5657, -1.0112,  3.0636,  2.8162, -1.1404, -1.0872, -0.2857,  1.8776,
         2.6911, -1.1517,  2.7511,  2.8491,  1.9049,  2.2861,  1.2595,  0.0375,
         2.2999, -1.0186,  2.8122,  2.4728,  2.4826,  2.6429, -1.2372, -1.1642,
         1.9981,  2.9473,  2.6417,  2.7006,  2.3943,  2.7325,  2.0554, -0.1539,
         0.6540, -1.0606, -0.9838, -1.0876,  2.6368, -0.7493, -1.1622,  2.0139],
       grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1.,
        0., 1., 1., 1., 0., 1., 1., 1., 0., 1.])
tensor([-0.1578,  2.1098,  2.9130, 

Epoch: 1/10... Step: 84... Loss: 0.251874... Val Loss: 0.294689
tensor([ 0.2770,  3.0996,  3.2764,  3.3351,  2.9733,  1.6123,  3.2880,  3.1024,
         1.4958, -0.5173,  3.1017,  2.2425,  3.3370,  0.2518,  3.1556,  2.7671,
        -1.1250, -0.2356,  2.7424, -0.3089, -0.3092,  2.2537, -0.7075,  0.4120,
         2.3459,  0.0263,  0.1033,  1.7907,  2.9901,  2.4580, -0.8039,  1.9052,
         2.7730,  3.1875, -0.8228,  2.7898,  2.9904,  2.6574,  2.9315,  3.0601,
         3.0016,  2.6761,  2.0889,  3.1717, -0.7496,  2.9664,  2.3609,  3.0252,
         3.0231,  3.0391,  2.7531,  2.7686, -0.6988,  0.1806, -0.7279,  0.2653,
         3.1181,  2.3602, -0.7853,  2.6188, -0.6826,  3.0497,  2.6617,  2.6494],
       grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
        1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0.,
        0., 1., 1., 